In [ ]:
#importing libraries
import html2text
from bs4 import BeautifulSoup
from selenium import webdriver 
import pandas as pd 
import requests
from lxml import html
from selenium.webdriver.common.by import By
import numpy as np
import itertools
from itertools import zip_longest
import json
import time 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}

driver = webdriver.Chrome('')

#create api url from list of walletaddresses in excel sheet 
df = pd.read_excel()
df = df[df.Blockchain == 'BTC']
df.columns = df.columns.str.replace(" ","")
WAs= pd.DataFrame(df.WalletAddress)
WAs = 'https://blockstream.info/api/address/' + WAs.astype(str) + '/txs'
urls = pd.Series.tolist(WAs["WalletAddress"])
urls

#since api only allows scraping of last 25 transactions
longurlslist = []
lasturl =[]
for url in [urls[1]]:
    longurls = []
    count = 0 
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}
    page = requests.get(url, headers = headers).text
    response = json.loads(page)
    if len(response) > 24: 
        longurls.append(str(url) + '/chain/' + str(response[len(response)-1]['txid'])) 
        for longurl in longurls: 
            longpage = requests.get(longurls[count], headers = headers).text
            time.sleep(2)
            longresponse = json.loads(longpage)
            #check if there are more than 24 transactions, so we can catch the 25th url and create another API request using that txhash
            if len(longresponse) > 24: 
                longurls.append(str(url) +'/chain/' + str(longresponse[len(longresponse)-1]['txid'])) 
                count = count + 1
                print(count)
            else: 
                lasturl.append(str(url) +'/chain/' + str(longresponse[len(longresponse)-1]['txid']))
                longurlslist.extend(longurls)
                longurlslist.extend(lasturl)
                print('end')

    else:
        print('end2')
    #create long list of urls incl API requests for 25-50, 51-75 etc transactions
    longurlslist.extend([url])


#read JSON 
txnsum = []
for url in longurlslist:
    response = requests.get(url).text
    time.sleep(4)
    response_info = json.loads(response)
    data = response_info
    txnno = [i for i in range(len(data))]
    for i in txnno:
        txid = data[i]['txid']
        txndate = data[i]['status']['block_time']
        status = data[i]['status']['confirmed']
        fee = data[i]['fee']

        maxin = [x for x in range(len(data[i]['vin']))]
        maxout = [x for x in range(len(data[i]['vout']))]

        for x in maxin: 
            if data[i]['vin'][x]['prevout'].get('scriptpubkey_address'):
                inaddress = data[i]['vin'][x]['prevout']['scriptpubkey_address']
            else: 
                inaddress = 'error'
            if data[i]['vin'][x]['prevout'].get('value'):
                invalue = data[i]['vin'][x]['prevout']['value']
            else: 
                invalue = 'error'
            txnsum.append([txid, txndate, status, inaddress, invalue, 0, 0, fee])

        
        for x in maxout:
            if data[i]['vout'][x].get('scriptpubkey_address'):
                outaddress =data[i]['vout'][x]['scriptpubkey_address']
            else: 
                outaddress = 'OP Error'
            if data[i]['vout'][x].get('value'):
                outvalue = data[i]['vout'][x]['value']
            else: 
                outvalue = 'error'
            txnsum.append([txid, txndate, status, 0, 0, outaddress, outvalue, fee])

export = pd.DataFrame(txnsum)
export.to_csv('')

